<a href="https://colab.research.google.com/github/sundarbee/PythonWorkout/blob/main/house_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive

In [17]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
!pwd

/content


In [19]:
!ls


Dataset.zip  drive  sample_data


In [20]:
ls -l /content/drive/MyDrive/House_Prediction/

total 2129
-rw------- 1 root root 2179591 Sep 27 03:29 Dataset.zip


In [21]:
# copy the files to local
!cp /content/drive/MyDrive/House_Prediction/Dataset.zip .

In [22]:
!ls

Dataset.zip  drive  sample_data


In [ ]:
import zipfile
import pandas as pd


zip_path = '/content/Dataset.zip'
target_folder = 'Dataset/'  # Replace with the folder name inside the ZIP

# file_to_extract = 'Dataset/Train.csv'

with zipfile.ZipFile(zip_path, 'r') as z:
  dfs = {f:pd.read_csv(z.open(f), encoding='latin-1') for f in z.namelist() if f.endswith('.csv')}
train, test, sub = dfs['Dataset/Train.csv'], dfs['Dataset/Test.csv'], dfs['Dataset/sample_submission.csv']

train.head()



In [49]:
train.columns

Index(['area_type', 'availability', 'location', 'size', 'society',
       'total_sqft', 'bath', 'balcony', 'price'],
      dtype='object')

In [53]:
test

,area_type,availability,location,size,society,total_sqft,bath,balcony
0,Super built-up Area,Ready To Move,Brookefield,2 BHK,Roeekbl,1225,2.0,2.0
1,Plot Area,Ready To Move,Akshaya Nagar,9 Bedroom,NaN,2400,9.0,2.0
2,Plot Area,18-Apr,Hennur Road,4 Bedroom,Saandtt,1650,5.0,2.0
3,Super built-up Area,Ready To Move,Kodichikkanahalli,3 BHK,Winerri,1322,3.0,1.0
4,Super built-up Area,Ready To Move,Konanakunte,2 BHK,AmageSa,1161,2.0,1.0
...,...,...,...,...,...,...,...,...
1475,Super built-up Area,Ready To Move,Vittasandra,2 BHK,Prlla C,1246,2.0,1.0
1476,Super built-up Area,Ready To Move,Gottigere,3 BHK,NaN,1660,3.0,2.0
1477,Super built-up Area,Ready To Move,Channasandra,2 BHK,Unm 2El,1216,2.0,2.0
1478,Built-up Area,18-Feb,Tumkur Road,2 BHK,Sahtsva,996,2.0,1.0


In [54]:
sub

,price
0,0
1,0
2,0
3,0
4,0
...,...
1475,0
1476,0
1477,0
1478,0


In [58]:
train.shape,test.shape,sub.shape

((13320, 9), (1480, 8), (1480, 1))

In [60]:
# to find column diff between two DF
train.columns.difference(test.columns)

Index(['price'], dtype='object')

In [63]:
# Check missing data
train.isna().sum()

,0
area_type,0
availability,0
location,1
size,16
society,5502
total_sqft,0
bath,73
balcony,609
price,0


In [66]:
# to find the percentage of null values
train.isna().sum()/train.shape[0]*100

,0
area_type,0.000000
availability,0.000000
location,0.007508
size,0.120120
society,41.306306
total_sqft,0.000000
bath,0.548048
balcony,4.572072
price,0.000000


In [67]:
# which cols are having highest null value percentage
(train.isna().sum()/train.shape[0]*100).sort_values(ascending=False)

,0
society,41.306306
balcony,4.572072
bath,0.548048
size,0.120120
location,0.007508
area_type,0.000000
availability,0.000000
total_sqft,0.000000
price,0.000000


In [69]:
#to find how many unique values in each column
train.nunique()


,0
area_type,4
availability,81
location,1305
size,31
society,2688
total_sqft,2117
bath,19
balcony,4
price,1994


In [70]:
# train
train['area_type'].unique()

array(['Super built-up  Area', 'Plot  Area', 'Built-up  Area',
       'Carpet  Area'], dtype=object)

In [75]:
for i in train.columns:
  print(train[i].nunique(),train[i],train[i].unique())

4 0        Super built-up  Area
1                  Plot  Area
2              Built-up  Area
3        Super built-up  Area
4        Super built-up  Area
                 ...         
13315          Built-up  Area
13316    Super built-up  Area
13317          Built-up  Area
13318    Super built-up  Area
13319    Super built-up  Area
Name: area_type, Length: 13320, dtype: object ['Super built-up  Area' 'Plot  Area' 'Built-up  Area' 'Carpet  Area']
81 0               19-Dec
1        Ready To Move
2        Ready To Move
3        Ready To Move
4        Ready To Move
             ...      
13315    Ready To Move
13316    Ready To Move
13317    Ready To Move
13318           18-Jun
13319    Ready To Move
Name: availability, Length: 13320, dtype: object ['19-Dec' 'Ready To Move' '18-May' '18-Feb' '18-Nov' '20-Dec' '17-Oct'
 '21-Dec' '19-Sep' '20-Sep' '18-Mar' '20-Feb' '18-Apr' '20-Aug' '18-Oct'
 '19-Mar' '17-Sep' '18-Dec' '17-Aug' '19-Apr' '18-Jun' '22-Dec' '22-Jan'
 '18-Aug' '19-Jan' '17-Jul' '1

In [79]:
#List alll thjose in the above format wherein the unique count is less than 30
for col in train.columns:
    unique_vals = train[col].unique()
    num_unique = len(unique_vals)

    if num_unique < 30:
        print(f'Column: {col}')
        print(f'Unique Count: {num_unique}')
        print(f'Unique Values: {unique_vals}')


Unique Count: 4
Unique Count: 20
Unique Count: 5


In [80]:
filtered = [
    (col, train[col].nunique(), train[col].unique())
    for col in train.columns
    if train[col].nunique() > 30
]

for col, count, uniques in filtered:
    print(f'Column: {col}')
    print(f'Unique Count: {count}')
    print(f'Unique Values: {uniques}')


Column: availability
Unique Count: 81
Unique Values: ['19-Dec' 'Ready To Move' '18-May' '18-Feb' '18-Nov' '20-Dec' '17-Oct'
 '21-Dec' '19-Sep' '20-Sep' '18-Mar' '20-Feb' '18-Apr' '20-Aug' '18-Oct'
 '19-Mar' '17-Sep' '18-Dec' '17-Aug' '19-Apr' '18-Jun' '22-Dec' '22-Jan'
 '18-Aug' '19-Jan' '17-Jul' '18-Jul' '21-Jun' '20-May' '19-Aug' '18-Sep'
 '17-May' '17-Jun' '21-May' '18-Jan' '20-Mar' '17-Dec' '16-Mar' '19-Jun'
 '22-Jun' '19-Jul' '21-Feb' 'Immediate Possession' '19-May' '17-Nov'
 '20-Oct' '20-Jun' '19-Feb' '21-Oct' '21-Jan' '17-Mar' '17-Apr' '22-May'
 '19-Oct' '21-Jul' '21-Nov' '21-Mar' '16-Dec' '22-Mar' '20-Jan' '21-Sep'
 '21-Aug' '14-Nov' '19-Nov' '15-Nov' '16-Jul' '15-Jun' '17-Feb' '20-Nov'
 '20-Jul' '16-Sep' '15-Oct' '15-Dec' '16-Oct' '22-Nov' '15-Aug' '17-Jan'
 '16-Nov' '20-Apr' '16-Jan' '14-Jul']
Column: location
Unique Count: 1305
Unique Values: ['Electronic City Phase II' 'Chikka Tirupathi' 'Uttarahalli' ...
 '12th cross srinivas nagar banshankari 3rd stage' 'Havanur extension

In [81]:
[(i,train[i].nunique(),train[i].unique()) for i in train.columns if train[i].nunique()<30]

[('area_type',
  4,
  array(['Super built-up  Area', 'Plot  Area', 'Built-up  Area',
         'Carpet  Area'], dtype=object)),
 ('bath',
  19,
  array([ 2.,  5.,  3.,  4.,  6.,  1.,  9., nan,  8.,  7., 11., 10., 14.,
         27., 12., 16., 40., 15., 13., 18.])),
 ('balcony', 4, array([ 1.,  3., nan,  2.,  0.]))]

In [86]:
'''
create a function to
  total unique values
  list of unique values
  top 10 value_counts
  top 10 value_counts %
'''
def chk_uniqueness(col):
  val_counts = train[col].value_counts()
  top_10_values = val_counts.head(10)
  top_10_values_pct = (top_10_values/train.shape[0])*100
  print(f"unique values in '{col}' : {train[col].nunique()}  ")
  print(f"All unique values in '{col}' : {train[col].unique()}  ")
  print(f"Top 10 value counts in '{col}' : {top_10_values}  ")
  print(f"Top 10 value counts % in '{col}' : {top_10_values_pct}  ")

In [87]:
chk_uniqueness('area_type')

unique values in 'area_type' : 4  
All unique values in 'area_type' : ['Super built-up  Area' 'Plot  Area' 'Built-up  Area' 'Carpet  Area']  
Top 10 value counts in 'area_type' : area_type
Super built-up  Area    8790
Built-up  Area          2418
Plot  Area              2025
Carpet  Area              87
Name: count, dtype: int64  
Top 10 value counts % in 'area_type' : area_type
Super built-up  Area    65.990991
Built-up  Area          18.153153
Plot  Area              15.202703
Carpet  Area             0.653153
Name: count, dtype: float64  
